### 1. Read in data 

In [1]:
%matplotlib notebook
from scipy.optimize import leastsq, brent
from scipy.linalg import solve_triangular
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=16)

def trap(vec, dx):
    # Perform trapezoidal integration
    return dx * (np.sum(vec) - 0.5 * (vec[0] + vec[-1]))

def func_to_min(h, x0, M, W):
    N = len(h)
    nu = (np.arange(M, dtype=float) + 0.5) / (2 * M)
    x = x0 * np.arange(N+1, dtype=float)/N
    C = calc_gridder_as_C(h, x0, nu, W)
    dnu = nu[1] - nu[0]
    dx = x[1] - x[0]
    h_ext = np.concatenate(([1.0], h))
    loss = np.zeros((len(h_ext), 2, M), dtype=float)
    for n, x_val in enumerate(x):
        one_app = 0
        for r in range(0, W):
            l = r - (W/2) + 1
            one_app += h_ext[n] * C[r, :] * np.exp(2j * np.pi * (l - nu) * x_val)
        loss[n, 0, :] = 1.0 - np.real(one_app)
        loss[n, 1, :] = np.imag(one_app)
        if n in [0, N]:
            loss[n, :, :] /= np.sqrt(2)
    loss = loss.reshape(2 * M * (N + 1))
    return loss

def optimal_grid(W, x0, N, M, h_initial=None):
    if h_initial is None:
        h_initial = np.ones(N, dtype=float)
    return leastsq(func_to_min, h_initial, args=(x0, M, W), full_output=True)

def make_evaluation_grids(W, M, N):
    """Generate vectors nu and x on which the gridder and gridding correction functions need to be evaluated.
        W is the number of integer gridpoints in total 
        M determines the sampling of the nu grid, dnu = 1/(2*M)
        N determines the sampling of the x grid, dx = 1/(2*N)
    """
    nu = (np.arange(W * M, dtype=float) + 0.5) / (2 * M)
    x = np.arange(N+1, dtype=float)/(2 * N)
    return nu, x

def calc_gridder_as_C_old(h, x0, nu, R):
    # Calculate gridding function C(l,nu) from gridding correction h(x) evaluated at (n+1) x_0/N where n is in range 0,...,N-1. 
    #  We assume that h(0)=1 for normalization, and use the trapezium rule for integration.
    # The gridding function is calculated for l=-R+1 to R at points nu
    M = len(nu)
    C = np.zeros((W, M), dtype=float)
    K = np.zeros((W, W))
    N = len(h)
    x = x0 * np.arange(0, N+1, dtype=float)/N
    dx = x0 / N
    h_ext = np.concatenate(([1.0], h))
    
    for rp in range(0, W):
        lp = rp - (W/2) + 1
        for r in range(W):
            l = r -  (W/2) + 1
            K[rp, r] = trap((h_ext**2) * np.cos(2 * np.pi * (lp - l) * x), dx)
    #Kinv = np.linalg.pinv(K)
    
    rhs = np.zeros(W, dtype=float)
    for m, nu_val in enumerate(nu):
        for rp in range(0, W):
            lp = rp - (W/2) + 1
            rhs[rp] = trap(h_ext * np.cos(2 * np.pi * (lp - nu_val) * x), dx)
        C[:,m] = np.linalg.lstsq(K, rhs, 1e-14)[0]
        # C[:,m] = np.dot(Kinv, rhs)
    return C

def calc_gridder_as_C(h, x0, nu, W):
    # Calculate gridding function C(l,nu) from gridding correction h(x) evaluated at (n+1) x_0/N where n is in range 0,...,N-1. 
    #  We assume that h(0)=1 for normalization, and use the trapezium rule for integration.
    # The gridding function is calculated for l=(1-W)/2 to (W-1)/2 at points nu
    factor = 1./np.sqrt(2.)
    M = len(nu)
    C = np.zeros((W, M), dtype=float)
    N = len(h)
    B = np.zeros((2 * N + 2, W))
    x = x0 * np.arange(0, N+1, dtype=float)/N
    dx = x0 / N
    h_ext = np.concatenate(([1.0], h))
    
    rhs = np.r_[np.ones(N + 1, dtype=float), np.zeros(N + 1, dtype=float)]
    rhs[0] = rhs[0]*factor
    rhs[N] = rhs[N]*factor

    for m, nu_val in enumerate(nu):
        for r in range(W):
            k = r - (W/2) + 1
            B[:N+1, r] = h_ext * np.cos(2 * np.pi * (k - nu_val) * x)
            B[N+1:, r] = h_ext * np.sin(2 * np.pi * (k - nu_val) * x)
        B[0, :] = B[0, :]*factor
        B[N, :] = B[N, :]*factor
        B[N+1, :] = B[N+1, :]*factor
        B[2*N+1, :] = B[2*N+1, :]*factor
#        q, r = np.linalg.qr(B)
#        C[:,m] = solve_triangular(r,np.dot(q.transpose(),rhs))
        C[:,m] = np.linalg.lstsq(B, rhs)[0]
    return C

def calc_gridder_as_C_odd(h, x0, nu, W):
    # Calculate gridding function C(l,nu) from gridding correction h(x) evaluated at (n+1) x_0/N where n is in range 0,...,N-1. 
    #  We assume that h(0)=1 for normalization, and use the trapezium rule for integration.
    # The gridding function is calculated for l=(1-W)/2 to (W-1)/2 at points nu
    factor = 1./np.sqrt(2.)
    M = len(nu)
    C = np.zeros((W, M), dtype=float)
    N = len(h)
    B = np.zeros((2 * N + 2, W))
    x = x0 * np.arange(0, N+1, dtype=float)/N
    dx = x0 / N
    h_ext = np.concatenate(([1.0], h))
    
    rhs = np.r_[np.ones(N + 1, dtype=float), np.zeros(N + 1, dtype=float)]
    rhs[0] = rhs[0]*factor
    rhs[N] = rhs[N]*factor

    for m, nu_val in enumerate(nu):
        for r in range(W):
            if nu_val > 0.5:
                k = r - (W//2) + 1
            else:
                k = r - (W//2) 
            B[:N+1, r] = h_ext * np.cos(2 * np.pi * (k - nu_val) * x)
            B[N+1:, r] = h_ext * np.sin(2 * np.pi * (k - nu_val) * x)
        B[0, :] = B[0, :]*factor
        B[N, :] = B[N, :]*factor
        B[N+1, :] = B[N+1, :]*factor
        B[2*N+1, :] = B[2*N+1, :]*factor
#        q, r = np.linalg.qr(B)
#        C[:,m] = solve_triangular(r,np.dot(q.transpose(),rhs))
        C[:,m] = np.linalg.lstsq(B, rhs)[0]
    return C


def calc_gridder(h, x0, nu, W):
    # Calculate gridder function on a grid nu which should have been generated using make_evaluation_grids
    #  The array h is the result of an optimization process for the gridding correction function evaluated
    #  on a relatively coarse grid extending from 0 to x0
    C = calc_gridder_as_C(h, x0, nu, W)
    gridder = np.zeros(M*W, dtype=float)
    for m in range(M):
        for rp in range(0, W):
            lp = rp - (W/2) + 1
            indx = int(m - 2*lp*M)
            if indx >= 0:
                gridder[indx] = C[rp, m]
            else:
                gridder[-indx-1] = C[rp, m]
    return gridder

def gridder_to_C(gridder, W):
    """Reformat gridder evaluated on the nu grid returned by make_evaluation_grids into the sampled C function 
    which has an index for the closest gridpoint and an index for the fractional distance from that gridpoint
    """
    M = len(gridder) // W
    C = np.zeros((W, M), dtype=float)
    for r in range(0, W):
        l = r - (W/2) + 1
        indx = (np.arange(M) - 2 * M * l).astype(int)
        # Use symmetry to deal with negative indices
        indx[indx<0] = -indx[indx<0] - 1
        C[r, :] = gridder[indx]
    return C
    
def gridder_to_grid_correction(gridder, nu, x, W):
    """Calculate the optimal grid correction function from the gridding function. The vectors x and nu should
    have been constructed using make_evaluation_grids"""
    M = len(nu) // W
    N = len(x) - 1
    dnu = nu[1] - nu[0]
    C = gridder_to_C(gridder, W)
    c = np.zeros(x.shape, dtype=float)
    d = np.zeros(x.shape, dtype=float)
    for n, x_val in enumerate(x):
        for rp in range(0, W):
            lp = rp - (W/2) + 1
            for r in range(0, W):
                l = r - (W/2) + 1
                d[n] += np.sum(C[rp, :] * C[r, :] * np.cos(2 * np.pi * (lp - l) * x_val)) * dnu
            c[n] += np.sum(C[rp, :] * np.cos(2 * np.pi * (lp - nu[:M]) * x_val)) * dnu
    return c/d

def calc_map_error(gridder, grid_correction, nu, x, W):
    M = len(nu) // W
    N = len(x) - 1
    dnu = nu[1] - nu[0]
    C = gridder_to_C(gridder, W)
    loss = np.zeros((len(x), 2, M), dtype=float)
    for n, x_val in enumerate(x):
        one_app = 0
        for r in range(0, W):
            l = r - (W/2) + 1
            one_app += grid_correction[n] * C[r, :] * np.exp(2j * np.pi * (l - nu[:M]) * x_val)
        loss[n, 0, :] = 1.0 - np.real(one_app)
        loss[n, 1, :] = np.imag(one_app)
    map_error = np.zeros(len(x), dtype=float)
    for i in range(len(x)):
        map_error[i] = 2 * np.sum((loss[i, :, :].flatten())**2) * dnu
    return map_error

In [2]:
data = np.genfromtxt('out_barray_6d.csv', delimiter = ',')
jj = complex(0,1)
u = data.T[0]
v = data.T[1]
w = -data.T[2]
V = data.T[3] + jj*data.T[4]
n_uv = len(u)
uv_max = max(np.sqrt(u**2+v**2))
n_uv

46332

In [3]:
for i in range(n_uv):
    #print (i,w[i])
    if w[i] < 0:
        u[i] = -u[i]
        v[i] = -v[i]
        w[i] = -w[i]
        V[i] = np.conjugate(V[i])

### 2. determine the number of w planes 

In [4]:
l_size = 900
m_size = 900
im_size = 1800
l_min = -np.pi/60.
l_max = np.pi/60.
l = np.linspace(l_min, l_max, num=l_size+1)[0:l_size]
m_min = -np.pi/60.
m_max = np.pi/60.
m = np.linspace(m_min,m_max,num=m_size+1)[0:m_size]
pixel_resol_l = 180. * 60. * 60. * (l_max - l_min) / np.pi / l_size
pixel_resol_m = 180. * 60. * 60. * (m_max - m_min) / np.pi / m_size
pixel_resol_l

23.999999999999996

### w plane number N_w


In [5]:
x0 = 0.25
n_min = (1-np.sqrt(1-(l_max)**2-(m_max)**2))
dw = x0/n_min
print (n_min,dw)


0.0027453251832613157 91.06389345943052


In [6]:
w_max = max(w)
w_min = min(w)
print (w_max, w_min)

2676.851655066061 5.4998167697155266e-05


In [7]:
N_w = int(np.ceil(w_max/dw))
N_w

30

### 3. w-stacking

#### 3-d Gridding

In [8]:
h = np.array([1.0004538705524384, 1.0018167571888508, 1.0040924910037743,
       1.0072874777218557, 1.0114107286991567, 1.01647390462776  ,
       1.0224913722498286, 1.0294802744795768, 1.0374606144276517,
       1.0464553539230763, 1.056490527234162 , 1.0675953708026737,
       1.0798024699261606, 1.093147923452921 , 1.1076715276939142,
       1.1234169809074428, 1.140432109877257 , 1.1587691202845547,
       1.1784848727711617, 1.1996411868070795, 1.2223051747130087,
       1.2465496084500247, 1.2724533220773244, 1.3001016530981768,
       1.329586926267655 , 1.3610089838276185, 1.39447576656945  ,
       1.4301039506085578, 1.4680196452926424, 1.5083591582649176,
       1.5512698343715243, 1.5969109758478965, 1.6454548520515748,
       1.6970878079399683, 1.75201148153333  , 1.8104441417699515,
       1.8726221594683596, 1.9388016255784695, 2.0092601325509496,
       2.084298736505226 , 2.1642441199585236, 2.249450977221874 ,
       2.340304647209209 , 2.43722402138255  , 2.5406647579157084,
       2.6511228369532525, 2.7691384961307297, 2.8953005903750744,
       3.0302514254996726, 3.1746921213372645, 3.3293885672199095,
       3.495178040638391 , 3.6729765690316256, 3.8637871250289186,
       4.068708757277644 , 4.288946772448109 , 4.525824099360779 ,
       4.780793983712307 , 5.05545418191414  , 5.351562845481708 ,
       5.671056313665354 , 6.016069062109771 , 6.388956089864428 ,
       6.792318066738979 ])


In [9]:
Nfft = 1800
W = 14
R = W//2
x0 = 0.25 
M = 32
w_values = [(w_max - w_min)/N_w*i for i in range(-R+1, N_w+R+1)]
dw = w_values[2]-w_values[1]
Nw_2R = len(w_values)
Nw_2R

44

In [10]:
def nearest_wlayer(w_values,w,dw):
    #find the index of the w plane for each w value
    ind = []
    for i in range(n_uv):
        for j in range(len(w_values)):
            diff = w[i]-w_values[j]
            if diff >= 0 and diff <= dw:
                ind += [j]
    return ind

ind = nearest_wlayer(w_values,w,dw)


In [11]:
print (w_max, max(w_values))

2676.851655066061 3301.4503067504015


#### Gridding on w

In [12]:
bEAM = np.ones(n_uv)
V_wgrid = np.zeros((Nw_2R,1),dtype = np.complex_).tolist()
beam_wgrid = np.zeros((Nw_2R,1),dtype = np.complex_).tolist()
u_wgrid = np.zeros((Nw_2R,1),dtype = np.complex_).tolist()
v_wgrid = np.zeros((Nw_2R,1),dtype = np.complex_).tolist()
for w_i in range(n_uv):
    tempw = (w[w_i] - w_values[ind[w_i]])/dw
    #print (tempw)
    w_plane = ind[w_i]
    C_w = calc_gridder_as_C(h, x0, np.asarray([tempw]), W)
    for n in range(-R+1,R+1):
        #print (tempw,w_i,w_plane,w_plane+n)
        V_wgrid[w_plane+n] += [C_w[n+R-1,0] * V[w_i]]
        u_wgrid[w_plane+n] += [u[w_i]]
        v_wgrid[w_plane+n] += [v[w_i]]
        beam_wgrid[w_plane+n] += [C_w[n+R-1,0] * bEAM[w_i]]

<ipython-input-1-682a474dd9d7>:104: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  C[:,m] = np.linalg.lstsq(B, rhs)[0]


In [13]:
for i in range(Nw_2R):
    del(V_wgrid[i][0])
    del(u_wgrid[i][0])
    del(v_wgrid[i][0])
    del(beam_wgrid[i][0])

In [15]:
I_image = np.zeros((Nw_2R,im_size,im_size),dtype = np.complex_)
B_image = np.zeros((Nw_2R,im_size,im_size),dtype = np.complex_)
B_grid = np.zeros((Nw_2R,im_size,im_size),dtype = np.complex_) 
V_grid = np.zeros((Nw_2R,im_size,im_size),dtype = np.complex_)
grid_num = im_size//2
ugrid_num = grid_num
vgrid_num = grid_num
R = W//2
for w_ind in range(Nw_2R):
    print ('This is the ', w_ind, 'th level facet out of ',Nw_2R,' w facets.\n')
    V_update = np.asarray(V_wgrid[w_ind])
    u_update = np.asarray(u_wgrid[w_ind])
    v_update = np.asarray(v_wgrid[w_ind])
    beam_update = np.asarray(beam_wgrid[w_ind])
    u_grid = u_update * 2 * (l_max - l_min) + im_size//2
    v_grid = v_update * 2 * (m_max - m_min) + im_size//2
    #bEAM = np.ones(len(V_update))
    for k in range(0,len(V_wgrid[w_ind])):
        tempu = u_grid[k]
        tempv = v_grid[k]
        for i in range(0, im_size):
            for j in range(0, im_size):
                if tempu>=i and tempu<i+1 and tempv>=j and tempv<j+1:
                    tempu = tempu - i
                    tempv = tempv - j
                    C_u = calc_gridder_as_C(h, x0, np.asarray([tempu]), W)
                    C_v = calc_gridder_as_C(h, x0, np.asarray([tempv]), W)
                    for mm in range(-R+1,R+1):
                        for n in range(-R+1,R+1):
                            V_grid[w_ind,i+mm,j+n] += C_u[mm+R-1,0] * C_v[n+R-1,0] * V_update[k]
                            B_grid[w_ind,i+mm,j+n] += C_u[mm+R-1,0] * C_v[n+R-1,0] * beam_update[k] 


    II = np.fft.ifftshift(V_grid[w_ind])
    II = np.fft.ifftn(II)
    II = np.fft.ifftshift(II)
    I_image[w_ind] = II
    SB = np.fft.ifftshift(B_grid[w_ind])
    SB = np.fft.ifftn(SB)
    SB = np.fft.ifftshift(SB)
    B_image[w_ind] = SB


This is the  0 th level facet out of  44  w facets.



<ipython-input-1-682a474dd9d7>:104: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  C[:,m] = np.linalg.lstsq(B, rhs)[0]


KeyboardInterrupt: 

In [ ]:
n_v = []
for w_ind in range(Nw_2R):
    n_v += [len(V_wgrid[w_ind])]



In [ ]:
I_image.shape

In [ ]:
plt.figure()
plt.imshow(np.rot90(I_image[4].real,1), origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
B_image[4,900,900]*im_size*im_size/n_v[4]

In [ ]:
I_image_sum_pre = np.zeros((im_size,im_size),dtype = np.complex_)
B_image_sum_pre = np.zeros((im_size,im_size),dtype = np.complex_)
aa = 0
for w_ind in range(Nw_2R):
    I_image_sum_pre += I_image[w_ind]
    B_image_sum_pre += B_image[w_ind]

I_image_sum_pre = I_image_sum_pre * im_size * im_size / n_uv 
B_image_sum_pre = B_image_sum_pre * im_size * im_size / n_uv 
plt.figure()
plt.imshow(np.rot90(I_image_sum_pre.real,1), origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
B_image_sum_pre[300,300]

In [ ]:
B_image_sum_pre[900,900]

In [ ]:
n_v = []
for w_ind in range(Nw_2R):
    n_v += [len(V_wgrid[w_ind])]
   

In [ ]:
I_image_corrected = np.zeros([Nw_2R,l_size,m_size],dtype = np.complex_)
B_image_corrected = np.zeros([Nw_2R,l_size,m_size],dtype = np.complex_)
I_image_corrected1 = np.zeros([Nw_2R,l_size,m_size],dtype = np.complex_)
B_image_corrected1 = np.zeros([Nw_2R,l_size,m_size],dtype = np.complex_)
index_x = int(l_size * 1.5)
index_y = int(m_size * 1.5)
for w_size in range(Nw_2R):
    temp = np.delete(B_image[w_size],np.s_[0:l_size//2],0)
    temp = np.delete(temp,np.s_[l_size:index_x],0)
    temp = np.delete(temp,np.s_[0:m_size//2],1)
    B_image_corrected[w_size] = np.delete(temp,np.s_[m_size:index_y],1)
    temp = np.delete(I_image[w_size],np.s_[0:l_size//2],0)
    temp = np.delete(temp,np.s_[l_size:index_x],0)
    temp = np.delete(temp,np.s_[0:m_size//2],1)
    I_image_corrected[w_size] = np.delete(temp,np.s_[m_size:index_y],1)

In [ ]:
I_image1 = np.zeros((Nw_2R,im_size//2,im_size//2),dtype = np.complex_)
B_image1 = np.zeros((Nw_2R,im_size//2,im_size//2),dtype = np.complex_)
for w_ind in range(Nw_2R):
    #print ('This is the ', w_ind, 'th level facet out of ',N_w,' w facets.\n')    
    for l_i in range(0,l_size):
        for m_i in range(0,m_size):
            ll = l[l_i]
            mm = m[m_i]
            nn = np.sqrt(1 - ll**2 - mm**2)-1
            I_image1[w_ind,l_i,m_i] = np.exp(jj*2*np.pi*(w_values[w_ind])*nn)*I_image_corrected[w_ind,l_i,m_i]
            B_image1[w_ind,l_i,m_i] = np.exp(jj*2*np.pi*(w_values[w_ind])*nn)*B_image_corrected[w_ind,l_i,m_i]
    

In [ ]:
plt.figure()
plt.imshow(np.rot90(I_image1[4].real,1), origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
B_image1[4,450,450]*im_size*im_size/n_v[4]

In [ ]:
I_image_sum = np.zeros((im_size//2,im_size//2),dtype = np.complex_)
B_image_sum = np.zeros((im_size//2,im_size//2),dtype = np.complex_)
for w_ind in range(Nw_2R):
    I_image_sum += I_image1[w_ind]
    B_image_sum += B_image1[w_ind]

I_image_sum = I_image_sum*im_size*im_size/n_uv
B_image_sum = B_image_sum*im_size*im_size/n_uv

np.savetxt('I_FFT_900_W-stacking-improved-withoutcorrecting_W14.csv', I_image_sum.real, delimiter = ',')
np.savetxt('B_FFT_900_W-stacking-improved-withoutcorrecting_W14.csv', B_image_sum.real, delimiter = ',')

In [ ]:
plt.figure()
plt.imshow(np.rot90(I_image_sum.real,1), origin = 'lower', cmap='gray')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
plt.savefig('W-stacking-improved-withoutcorrecting.png', bbox_inches = 'tight')
plt.show()
B_image_sum[450,450]

#### Correcting functions h(x)h(y)

#### R = 7 x0 = 0.25

In [ ]:
Nfft = 1800
x0 = 0.25
W = 14
# Use these for calculating gridding correction on the FFT grid
M = 32
nu, x = make_evaluation_grids(W, M, Nfft/2)
gridder = calc_gridder(h, x0, nu, W)
grid_correction4 = gridder_to_grid_correction(gridder, nu, x, W)
h_map4 = np.zeros(Nfft, dtype=float)
h_map4[Nfft//2:] = grid_correction4[:Nfft//2]
h_map4[:Nfft//2] = grid_correction4[:0:-1]
temp = np.delete(h_map4,np.s_[0:l_size//2],0)
index_x = int(l_size * 1.5)
index_y = int(m_size * 1.5)
Cor_gridx = np.delete(temp,np.s_[l_size:index_x],0)
Cor_gridy = np.delete(temp,np.s_[m_size:index_y],0)

In [ ]:
I_image_sum_corrected = np.zeros([l_size,m_size],dtype = np.complex_)
B_image_sum_corrected = np.zeros([l_size,m_size],dtype = np.complex_)

for i in range(0,im_size//2):
    for j in range(0,im_size//2):
        B_image_sum_corrected[i,j] = B_image_sum[i,j] * Cor_gridx[i] * Cor_gridy[j]

for i in range(0,im_size//2):
    for j in range(0,im_size//2):
        I_image_sum_corrected[i,j] = I_image_sum[i,j] * Cor_gridx[i] * Cor_gridy[j]


In [ ]:
plt.figure()
plt.imshow(np.rot90(I_image_sum_corrected.real,1), origin = 'lower',cmap='gray')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
plt.savefig('W-stacking-improved-withxycorrecting.png', bbox_inches = 'tight')
plt.show()
plt.figure()
plt.imshow(np.rot90(B_image_sum_corrected.real,1), origin = 'lower',cmap='gray')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
plt.savefig('W-stacking-improved-withxycorrecting_beam.png', bbox_inches = 'tight')
plt.show()
B_image_sum_corrected[450,450]
np.savetxt('I_FFT_900_W-stacking-improved-withxycorrecting_W14.csv', I_image_sum_corrected.real, delimiter = ',')
np.savetxt('B_FFT_900_W-stacking-improved-withxycorrecting_W14.csv', B_image_sum_corrected.real, delimiter = ',')

In [ ]:
def RMS(y):
    return np.sqrt(np.mean(y**2))

I_DFT = np.loadtxt('I_DFT_900_out6db.csv', delimiter = ',')
B_DFT = np.loadtxt('B_DFT_900_out6db.csv', delimiter = ',')
print (RMS(I_image_sum_corrected.real - I_DFT),RMS(B_image_sum_corrected.real - B_DFT))
plt.figure()
plt.imshow(np.rot90(I_image_sum_corrected.real - I_DFT,1), origin = 'lower',cmap='gray')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
plt.savefig('w-stacking-difference-xycorrecting.png', bbox_inches = 'tight',)
plt.show()
plt.figure()
plt.imshow(np.rot90(B_image_sum_corrected.real - B_DFT,1), origin = 'lower',cmap='gray')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
B_image_sum_corrected[450,450]

#### Correcting function on z axis

In [ ]:
def int5(h_x, iz, zin, step):
    y0=h_x[iz]
    y1=h_x[iz+step]
    ym1=h_x[iz-step]
    y2=h_x[iz+2*step]
    y3=h_x[iz+3*step]
    ym2=h_x[iz-2*step]
    a0=y0
    if((zin<0.) or (zin>1.)):
        print("This should not happen\n.")
    else:
        a1 = y1-(1./3)*y0-(1./2)*ym1+(1./20)*ym2-(1./4)*y2+(1./30)*y3
        a2 = (2./3)*ym1-(5./4)*y0+(2./3)*y1-(1./24)*ym2-(1./24)*y2
        a3 = (7./24)*y2+(5./12)*y0-(7./12)*y1-(1./24)*ym2-(1./24)*ym1-(1./24)*y3
        a4 = (1./24)*ym2+(1./4)*y0-(1./6)*y1-(1./6)*ym1+(1./24)*y2
        a5 = (1./120)*y3-(1./12)*y0+(1./12)*y1+(1./24)*ym1-(1./24)*y2-(1./120)*ym2 
        ans = a0 + a1*zin + a2*zin*zin + a3*zin*zin*zin + a4*zin*zin*zin*zin +a5*zin*zin*zin*zin*zin
    return ans



In [ ]:
yrange = np.pi/60.*2
xrange = np.pi/60.*2
ny = 1800
nx = 1800
dw = w_values[1] - w_values[0]
fmap = np.zeros((nx,ny))
for i in range(ny):
    yy = 2.*yrange*(i - ny/2)/ny
    for j in range(nx):
        xx = 2.*xrange*(j - nx/2)/nx
        if (xx*xx + yy*yy > 0.99999999) or (abs(xx) > 0.55*xrange) or (abs(yy) > 0.55*yrange):
            z = 0.
        else:
            z = dw*(1. - np.sqrt(1. - xx*xx - yy*yy))
            ind0 = (int)(z*nx + nx/2.)
            #print (z, ind0)
            xin = (float) (z*nx + nx/2.) - ind0
            fmap[i,j] = int5(h_map4,ind0,xin,1)

In [ ]:
plt.figure()
plt.imshow(fmap, origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
fmap[900,900]

In [ ]:
temp = np.delete(fmap,np.s_[0:l_size//2],0)
temp = np.delete(temp,np.s_[l_size:index_x],0)
temp = np.delete(temp,np.s_[0:m_size//2],1)
Cor_gridz = np.delete(temp,np.s_[m_size:index_y],1)

In [ ]:
plt.figure()
plt.imshow(Cor_gridz, origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
Cor_gridz[450,450]

In [ ]:
I_image_sum_corrected = np.loadtxt('I_FFT_900_W-stacking-improved-withxycorrecting_W14.csv', delimiter = ',')
B_image_sum_corrected = np.loadtxt('B_FFT_900_W-stacking-improved-withxycorrecting_W14.csv', delimiter = ',')
I_image_sum_zcorrected = np.zeros([l_size,m_size],dtype = np.complex_)
B_image_sum_zcorrected = np.zeros([l_size,m_size],dtype = np.complex_)

for i in range(0,im_size//2):
    for j in range(0,im_size//2):
        B_image_sum_zcorrected[i,j] = B_image_sum_corrected[i,j] * Cor_gridz[i,j]

for i in range(0,im_size//2):
    for j in range(0,im_size//2):
        I_image_sum_zcorrected[i,j] = I_image_sum_corrected[i,j] * Cor_gridz[i,j]


In [ ]:
plt.figure()
plt.imshow(np.rot90(I_image_sum_zcorrected.real,1), origin = 'lower',cmap='gray')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
plt.savefig('W-stacking-improved-withxyzcorrecting.png', bbox_inches = 'tight')
plt.show()
B_image_sum_zcorrected[450,450]
np.savetxt('I_FFT_900_W-stacking-improved-withxyzcorrecting_W14.csv', I_image_sum_zcorrected.real, delimiter = ',')
np.savetxt('B_FFT_900_W-stacking-improved-withxyzcorrecting_W14.csv', B_image_sum_zcorrected.real, delimiter = ',')

In [ ]:
print (RMS(I_image_sum_zcorrected.real - I_DFT),RMS(B_image_sum_zcorrected.real - B_DFT))
plt.figure()
plt.imshow(np.rot90(I_image_sum_zcorrected.real - I_DFT,1), origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
plt.figure()
plt.imshow(np.rot90(B_image_sum_zcorrected.real - B_DFT,1), origin = 'lower')
plt.xlabel('Image Coordinates X')
plt.ylabel('Image Coordinates Y')
#plt.savefig('Zfaceting_DFT.png', bbox_inches = 'tight')
plt.show()
B_image_sum_corrected[450,450]

In [ ]:
RMS(I_image_sum_zcorrected.real- I_DFT)